# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import nltk
nltk.download('punkt')
nltk.download('wordnet')

import re
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [3]:
# load data from database
engine = create_engine('sqlite:///../db.sqlite3')
df = pd.read_sql_table('disasterdata', engine)
# X should be the message column and Y should be all of the other columns. So Y is multivariable or something?
X = np.ravel(df[["message"]].values)
column_names = list(df.iloc[:,4:100].columns)
y = df.iloc[:, 4:100].values

# actually, I'm thinking the above might be incorrect. I think I would have to create the "def load_data()" function
# something like:
# def load_data():
#     df = pd.read_csv('corporate_messaging.csv', encoding='latin-1')
#     df = df[(df["category:confidence"] == 1) & (df['category'] != 'Exclude')]
#     X = df.text.values
#     y = df.category.values
#     return X, y

# so basically I guess it's mainly the category confidence and exclude portions that I need to add?


In [27]:
column_names, y, range(36)

(['related',
  'request',
  'offer',
  'aid_related',
  'medical_help',
  'medical_products',
  'search_and_rescue',
  'security',
  'military',
  'child_alone',
  'water',
  'food',
  'shelter',
  'clothing',
  'money',
  'missing_people',
  'refugees',
  'death',
  'other_aid',
  'infrastructure_related',
  'transport',
  'buildings',
  'electricity',
  'tools',
  'hospitals',
  'shops',
  'aid_centers',
  'other_infrastructure',
  'weather_related',
  'floods',
  'storm',
  'fire',
  'earthquake',
  'cold',
  'other_weather',
  'direct_report'],
 array([[1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ..., 
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]]),
 range(0, 36))

In [28]:
range(column_names)

#column_names is a list of strings
#y_test is an array with numbers

TypeError: 'list' object cannot be interpreted as an integer

In [30]:
for i in column_names:
    print(i)
    c += 1
    print(c)
#     print(classification_report(y_test[:,i], y_test[:,i]))

related


NameError: name 'c' is not defined

In [33]:
for i in range(len(column_names)):
    print(column_names[i])
    print(classification_report(y_test[:,i], y_test[:,i]))

related
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      1526
          1       1.00      1.00      1.00      4978
          2       1.00      1.00      1.00        50

avg / total       1.00      1.00      1.00      6554

request
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5426
          1       1.00      1.00      1.00      1128

avg / total       1.00      1.00      1.00      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       1.00      1.00      1.00        26

avg / total       1.00      1.00      1.00      6554

aid_related
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3813
          1       1.00      1.00      1.00      2741

avg / total       1.00      1.00      1.00      6554

medical_help
             precision    recall  f1-sco

In [31]:
len(column_names)

36

### 2. Write a tokenization function to process your text data

In [17]:
regex_url = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

# want to use the message column and take the necessary steps to process it: normalize, tokenize, stop word removal, 
# and stem and lemmatize. 

# look at this page for the flow of the maching learning pipeline: https://learn.udacity.com/nanodegrees/nd025/parts/cd0018/lessons/ls12134/concepts/08e7389d-f63b-4c73-9242-9cfe0b89ae21

# I don't think there would be any URLs in the text, so we don't need the first bit of code?
def tokenize_custom(text):
    url_detect = re.findall(regex_url, text)
    for url in url_detect:
        text = text.replace(url, "place_holder_url")

    words = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_words = []
    for word in words:
        clean_word = lemmatizer.lemmatize(word).lower().strip()
        clean_words.append(clean_word)

    return clean_words


# X, y = load_data()
# for message in X[:5]:
#     tokens = tokenize(message)
#     print(message)
#     print(tokens, '\n')


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [18]:
# so down here, I think I need to replace the RandomForestClassifier() with MultiOutputClassifier()
# also, I don't think we call the pipeline fit or predict functions until steps 4 and 5 respectively? and then I recall
# the predict function in step 7?
# def main():
#     X, y = load_data()
#     X_train, X_test, y_train, y_test = train_test_split(X, y)

pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize_custom)),
    ('tefreq_indocfreq', TfidfTransformer()),
    ('classifier', MultiOutputClassifier(RandomForestClassifier()))
])





#     # display results
#     display_results(y_test, y_pred)


# main()

# yea I don't think we need to use feature union for this project. because we are only using the message column as input
# actually, might need to add to step 8

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [19]:
X.shape, y.shape

((26216,), (26216, 36))

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [48]:
y_train.shape

(19662, 36)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [49]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [50]:
y_pred.shape

(6554, 36)

In [51]:
y_test[:,0]

array([0, 0, 0, ..., 0, 1, 1])

In [52]:
# y and y_test are dataframes
# y_pred is an array.

# so the solution should be to make everything an array from the beginning. 
# but before doing that, create some new object storing the column names

In [53]:
range(36)

range(0, 36)

In [54]:
for i in range(36):
    print(column_names[i])
    print(classification_report(y_test[:,i], y_pred[:,i]))

related
             precision    recall  f1-score   support

          0       0.65      0.35      0.46      1548
          1       0.82      0.94      0.87      4955
          2       0.63      0.24      0.34        51

avg / total       0.78      0.80      0.77      6554

request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5404
          1       0.80      0.39      0.53      1150

avg / total       0.87      0.88      0.86      6554

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6517
          1       0.00      0.00      0.00        37

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.71      0.88      0.79      3824
          1       0.75      0.50      0.60      2730

avg / total       0.73      0.73      0.71      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [55]:
pipeline.get_params()

{'memory': None,
 'steps': [('vectorizer',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize_custom at 0x7f1fe7473488>,
           vocabulary=None)),
  ('tefreq_indocfreq',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('classifier',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
        

In [65]:
parameters = {
    'classifier__estimator__n_estimators': [5, 10],
    'classifier__estimator__min_samples_split': [2, 3]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3, n_jobs=8, cv=3)

In [66]:
cv.fit(X_train, y_train)


Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] classifier__estimator__min_samples_split=2, classifier__estimator__n_estimators=10 
[CV] classifier__estimator__min_samples_split=2, classifier__estimator__n_estimators=10 
[CV] classifier__estimator__min_samples_split=2, classifier__estimator__n_estimators=10 
[CV] classifier__estimator__min_samples_split=2, classifier__estimator__n_estimators=20 
[CV] classifier__estimator__min_samples_split=2, classifier__estimator__n_estimators=20 
[CV] classifier__estimator__min_samples_split=2, classifier__estimator__n_estimators=20 
[CV] classifier__estimator__min_samples_split=2, classifier__estimator__n_estimators=30 
[CV] classifier__estimator__min_samples_split=2, classifier__estimator__n_estimators=30 
[CV]  classifier__estimator__min_samples_split=2, classifier__estimator__n_estimators=10, score=0.21635642355813245, total= 6.7min
[CV] classifier__estimator__min_samples_split=2, classifier__estimator__n_estimators=30 
[CV]  cl

[Parallel(n_jobs=8)]: Done  10 out of  18 | elapsed: 19.7min remaining: 15.7min


[CV]  classifier__estimator__min_samples_split=3, classifier__estimator__n_estimators=10, score=0.2050656087885261, total= 6.2min
[CV]  classifier__estimator__min_samples_split=3, classifier__estimator__n_estimators=20, score=0.21895025938358254, total=10.5min
[CV]  classifier__estimator__min_samples_split=3, classifier__estimator__n_estimators=20, score=0.22978333841928594, total=10.6min
[CV]  classifier__estimator__min_samples_split=2, classifier__estimator__n_estimators=30, score=0.22657918828196522, total=16.7min
[CV]  classifier__estimator__min_samples_split=3, classifier__estimator__n_estimators=20, score=0.21101617332926456, total=10.2min
[CV]  classifier__estimator__min_samples_split=3, classifier__estimator__n_estimators=30, score=0.24092157461092462, total=12.3min
[CV]  classifier__estimator__min_samples_split=3, classifier__estimator__n_estimators=30, score=0.22703692401586817, total= 9.5min
[CV]  classifier__estimator__min_samples_split=3, classifier__estimator__n_estimator

[Parallel(n_jobs=8)]: Done  18 out of  18 | elapsed: 29.7min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=8,
       param_grid={'classifier__estimator__n_estimators': [10, 20, 30], 'classifier__estimator__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [1]:
y_pred_best = cv.predict(X_test)

NameError: name 'cv' is not defined

In [2]:
cv.best_params_

NameError: name 'cv' is not defined

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
for i in range(36):
    print(column_names[i])
    print(classification_report(y_test[:,i], y_pred_best[:,i]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# actually, we would add feature union to step 8 here potentially
# this would also be where we add custom transformers I'm assuming

### 9. Export your model as a pickle file

In [ ]:
import pickle

with open('final_model.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.